In [ ]:
# Deteccion de perros en timepo real

Se carga la unidad drive a usar

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Retinanet**
Es un popular detector de una sola etapa, que es preciso y funciona rápido. RetinaNet utiliza una red piramidal de características para detectar objetos de manera eficiente a múltiples escalas. 

In [ ]:
# Clonamos el repositorio de keras-retinanet
!git clone https://github.com/DavidReveloLuna/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


#Desinstalacion e instalacion de librerias

Desisntalamos las librerias por defecto para no generar conflicto de versiones

In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: tensorflow 2.1.0
Uninstalling tensorflow-2.1.0:
  Successfully uninstalled tensorflow-2.1.0
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-2.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.10.0


Instalamos librerias con las versiones que queremos

In [ ]:
!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install h5py==2.10.0

  Using cached tensorflow-2.1.0-cp37-cp37m-manylinux2010_x86_64.whl (421.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached h5py-3.4.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (4.1 MB)
  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 3.4.0
    Uninstalling h5py-3.4.0:
      Successfully uninstalled h5py-3.4.0


Ingresamos al directorio de retinanet

In [ ]:
cd keras-retinanet/

/content/keras-retinanet


Instalamos todas las librerias de python y usamos  el script setup.py

In [ ]:
# Instalación y configuración de keras-retinet
!pip install .
!python setup.py build_ext --inplace

Processing /content/keras-retinanet
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-linux_x86_64.whl size=170900 sha256=0683fd24af3f4122838d896462bd84848ba04ddc6facb990766643f5028ca869
  Stored in directory: /root/.cache/pip/wheels/32/29/34/9b33c07f08b1be9e77607c1fc6b08c679489aa7ddaed329652
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1
running build_ext
skipping 'keras_retinanet/utils/compute_

Importamos librerias que se hara uso en el cuadernilllo

In [ ]:
#
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image


In [ ]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

##Cargamos el modelo entrenado

Este modelo es el resultado de 10 epocas de entrenamiento con 200 pasos, el cual represento una menor perdida en el entrenamiento =0.69, luego procedemos a cargar el modelo con retinanet.

In [ ]:
# Con el modelo ya entrenado vamos a realizar predicciones
# Cargamos el modelo entrenado, y lo configuramos para que sea compatible con retinanet
#model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
model_path = "/content/drive/MyDrive/snapshots/resnet50_csv_10.h5"
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

/content/drive/MyDrive/snapshots/resnet50_csv_10.h5


Using TensorFlow backend.


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

Aqui cargamos el archivo que contiene las etiquetas de las clases que en nuestro caso es:
```
classes.csv :

dog 0
```

In [ ]:
# Cargamos el archivo que contiene las etiquetas de las clases /content/drive/MyDrive/Proyecto Detección de Mascotas/classes.csv
#labels_to_names = pd.read_csv('classes.csv', header=None).T.loc[0].to_dict()
labels_to_names = pd.read_csv('/content/drive/MyDrive/classes.csv', header=None).T.loc[0].to_dict()

Obtenemos la predicción del modelo: boxes, scores, labels

In [ ]:
import skimage.io as io
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)
  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )
  boxes /= scale
  return boxes, scores, labels

#Aqui cargamos el codigo para que podamos usar la webcam

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

Cargamos mas librerias

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import PIL
import io
import html
%matplotlib inline
#Librería para generar la alarma
from google.colab import output as son

Metodos para pasar el video leido con javascript a imagen en formato opencv

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

#Porcentaje de similitud
Funciones para comparar la similitud de las imagenes 

In [ ]:
from skimage.metrics import structural_similarity

#Módulo para comparar dos imágenes
def Orb_Sim(Img1,Img2):
    Orb=cv2.ORB_create()
    #Puntos y descriptores
    Puntos1,Descrip1=Orb.detectAndCompute(Img1,None)
    Puntos2,Descrip2=Orb.detectAndCompute(Img2,None)
    #Definimos el matcher
    BF=cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
    #Realizamos el match
    Matches=BF.match(Descrip1,Descrip2)
    #Buscamos las regiones similares
    RegionesSimilares=[I for I in Matches if I.distance<70]
    if(len(Matches)==0):
        return(0)
    return(len(RegionesSimilares)/len(Matches))
#Módulo para igualar las dimensiones de las imágenes
def Dimensiones(Img1,Img2):
    Sim,Dif=structural_similarity(Img1,Img2,full=True)
    return Sim
    #Fila inicial:final hasta columna inicial:final
def Recortar(Fi,Ff,Ci,Cf,Img):
    #Todos deben ser enteros
    ImgRecorte=Img[Fi:Ff,Ci:Cf]
    return(ImgRecorte)
def Comparar(Img1,Img2):
    Similitud=Orb_Sim(Img1,Img2)
    Porcentaje=round((Similitud*100),2)
    print(Porcentaje)
    if(Porcentaje>90):
        return(True)
    else:
        return(False)

#Mostramos el video en timepo real y hacemos la comparacion 
La comparacion se hace entre la entrada del foto del can perdido y el actual fotograma que muestra el video

In [ ]:
#LEER la imagen del perro a buscar
ImgPerroPerdido=cv2.imread("/content/negroPerdido.PNG",0)
#ImgPerroPerdido=cv2.imread("/content/pipoca2.jpg",0)
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    # --------------------------------------------------------------------
    boxes, scores, labels = predict(frame)
    draw = frame.copy()
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)            
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)       
            bbox_array = cv2.rectangle(bbox_array,(int(b[0]),int(b[1])),(int(b[0]+b[2]),int(b[1]+b[3])),(255,0,0),2)            
            ImgPerroEnVideo=Recortar(int(b[0]),int(b[3]),int(b[1]),int(b[2]),frame)     
            if(Comparar(ImgPerroEnVideo,ImgPerroPerdido)):
              son.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
              print("Te encontré")
              break
            else:              
              print("No eres tú")
    # --------------------------------------------------------------------
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes
        
camera.release()
cv2.destroyAllWindows()

MessageError: ignored